# Projet maison n° 3

In [6]:
# imports
import pandas as pd
import numpy as np

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [213]:
# pd.concat?
# chargement des données
yob_1880 = pd.read_csv('names/yob1881.txt',sep=',')
yob_1880.head()

,Mary,F,6919
0,Anna,F,2698
1,Emma,F,2034
2,Elizabeth,F,1852
3,Margaret,F,1658
4,Minnie,F,1653


In [214]:
import os, os.path

In [54]:
pd.concat?

In [ ]:
# path joining version for other paths
# DIR = '/tmp'
# print len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])

In [215]:
# names us
def df_names_us():
    df = pd.DataFrame()
    DIR = './names' #chemin d'accès au dossier
    files_list = [os.path.join(DIR, name) for name in os.listdir(DIR) if (os.path.isfile(os.path.join(DIR, name)) & os.path.join(DIR, name).endswith(".txt"))]
    data_frame_list = []
    for dir_name in files_list:
        df_yob = pd.read_csv(dir_name, sep=',', header=None).rename(columns = {0:'name', 1:'gender', 2:'births'}) # "year", "name", "gender", "births"
        df_yob['year'] = dir_name[11:15]
        df_yob = df_yob[["year", "name", "gender", "births"]]
        df_yob = df_yob.astype({"year": int, "name": str, "gender": str, "births": int})
        data_frame_list.append(df_yob)
    df = pd.concat(data_frame_list, ignore_index=True).sort_values(by=["year", "gender","births","name"], ascending = [True, False, False, True]).reset_index(drop=True)

    return df

In [216]:
df_names_us().head()

,year,name,gender,births
0,1880,John,M,9655
1,1880,William,M,9532
2,1880,James,M,5927
3,1880,Charles,M,5348
4,1880,George,M,5126


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [113]:
names_us_df = df_names_us().head()
names_us_df.dtypes

year       int64
name      object
gender    object
births     int64
dtype: object

In [114]:
names_us_df

,year,name,gender,births
1904418,1880,John,M,9655
1904419,1880,William,M,9532
1904420,1880,James,M,5927
1904421,1880,Charles,M,5348
1904422,1880,George,M,5126


In [116]:
df_fr.head()

,sexe,preusuel,annais,nombre
0,1,_PRENOMS_RARES,1900,1249
1,1,_PRENOMS_RARES,1901,1342
2,1,_PRENOMS_RARES,1902,1330
3,1,_PRENOMS_RARES,1903,1286
4,1,_PRENOMS_RARES,1904,1430


In [217]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    df_fr = pd.read_csv('nat2021.csv', sep=';')
    df_fr.rename(columns={'sexe':'gender', 'preusuel':'name', 'annais':'year', 'nombre':'births'}, inplace = True)
    df_fr['gender'].replace({1:'M', 2:'F'} , inplace = True)
    df_fr = df_fr[["year", "name", "gender", "births"]]
    df_fr = df_fr[(df_fr["year"]!='XXXX') & (df_fr["name"]!='_PRENOMS_RARES')]

    df_fr = df_fr.astype({"year": int, "name": str, "gender": str, "births": int})
    df_fr = df_fr[[len(name)>=2 for name in df_fr['name']]]
    df_fr['name'] = [name.lower().title() for name in df_fr['name']]
    df = df_fr.sort_values(by=["year", "gender","births","name"], ascending = [True, False, False, True]).reset_index(drop=True)

    return df

In [218]:
df_names_fr().head()

,year,name,gender,births
0,1900,Jean,M,14097
1,1900,Louis,M,9052
2,1900,Pierre,M,7456
3,1900,Joseph,M,7259
4,1900,Henri,M,6921


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [227]:
df_Webstat = pd.read_csv('Webstat_Export.csv', sep=';')
df_Webstat = df_Webstat.apply(lambda x: x.str.replace(',','.'))
df_Webstat.head(10)

,Titre :,Dollar australien (AUD),Lev bulgare (BGN),Real brésilien (BRL),Dollar canadien (CAD),Franc suisse (CHF),Yuan renminbi chinois (CNY),Livre chypriote (CYP),Couronne tchèque (CZK),Couronne danoise (DKK),...,Leu roumain (RON),Rouble russe (RUB),Couronne suédoise (SEK),Dollar de Singapour (SGD),Tolar slovène (SIT),Couronne slovaque (SKK),Baht thaïlandais (THB),Livre turque (TRY),Dollar des Etats-Unis (USD),Rand sud-africain (ZAR)
0,Code série :,EXR.D.AUD.EUR.SP00.A,EXR.D.BGN.EUR.SP00.A,EXR.D.BRL.EUR.SP00.A,EXR.D.CAD.EUR.SP00.A,EXR.D.CHF.EUR.SP00.A,EXR.D.CNY.EUR.SP00.A,EXR.D.CYP.EUR.SP00.A,EXR.D.CZK.EUR.SP00.A,EXR.D.DKK.EUR.SP00.A,...,EXR.D.RON.EUR.SP00.A,EXR.D.RUB.EUR.SP00.A,EXR.D.SEK.EUR.SP00.A,EXR.D.SGD.EUR.SP00.A,EXR.D.SIT.EUR.SP00.A,EXR.D.SKK.EUR.SP00.A,EXR.D.THB.EUR.SP00.A,EXR.D.TRY.EUR.SP00.A,EXR.D.USD.EUR.SP00.A,EXR.D.ZAR.EUR.SP00.A
1,Unité :,Dollar Australien (AUD),Lev Nouveau (BGN),Real Bresilien (BRL),Dollar Canadien (CAD),Franc Suisse (CHF),Yuan Ren Min Bi (CNY),Livre Cypriote (CYP),Couronne Tcheque (CZK),Couronne Danoise (DKK),...,Nouveau Ron (RON),Rouble Russe (RUB) (RUB),Couronne Suedoise (SEK),Dollar de Singapour (SGD),Tolar (SIT),Couronne Slovaque (SKK),Baht (THB),Nouvelle Livre Turque (TRY),Dollar des Etats-Unis (USD),Rand (ZAR)
2,Magnitude :,Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),...,Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0)
3,Méthode d'observation :,Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),...,Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A)
4,Source :,BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),...,BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0)
5,20/09/2022,1.4893,1.9558,5.2139,1.3268,0.9644,7.003,NaN,24.556,7.4368,...,4.934,NaN,10.8338,1.4074,NaN,NaN,36.968,18.2833,0.9986,17.7261
6,19/09/2022,1.495,1.9558,5.2886,1.3294,0.9658,7.0066,NaN,24.494,7.4373,...,4.93,NaN,10.7993,1.4082,NaN,NaN,36.983,18.2738,0.999,17.7267
7,18/09/2022,-,-,-,-,-,-,NaN,-,-,...,-,NaN,-,-,NaN,NaN,-,-,-,-
8,17/09/2022,-,-,-,-,-,-,NaN,-,-,...,-,NaN,-,-,NaN,NaN,-,-,-,-
9,16/09/2022,1.4894,1.9558,5.2279,1.3226,0.9579,6.9787,NaN,24.497,7.4366,...,4.9238,NaN,10.7541,1.4025,NaN,NaN,36.8,18.1923,0.9954,17.6004


In [146]:
# df_Webstat.info()

In [119]:
# lignes dupliquées
df_Webstat.duplicated().value_counts()

False    8666
dtype: int64

In [120]:
# chercher tous les lignes avec au moins un NaN
#df_Webstat.loc[df_Webstat.isna().any(axis=1)]

In [228]:
# suppression de toutes les lignes avec un NaN
df_Webstat = df_Webstat.dropna()

In [229]:
# renommer la colonne 'Titre'
df_Webstat = df_Webstat.rename(columns={'Titre :': 'dates'})
df_Webstat.head()

,dates,Dollar australien (AUD),Lev bulgare (BGN),Real brésilien (BRL),Dollar canadien (CAD),Franc suisse (CHF),Yuan renminbi chinois (CNY),Livre chypriote (CYP),Couronne tchèque (CZK),Couronne danoise (DKK),...,Leu roumain (RON),Rouble russe (RUB),Couronne suédoise (SEK),Dollar de Singapour (SGD),Tolar slovène (SIT),Couronne slovaque (SKK),Baht thaïlandais (THB),Livre turque (TRY),Dollar des Etats-Unis (USD),Rand sud-africain (ZAR)
0,Code série :,EXR.D.AUD.EUR.SP00.A,EXR.D.BGN.EUR.SP00.A,EXR.D.BRL.EUR.SP00.A,EXR.D.CAD.EUR.SP00.A,EXR.D.CHF.EUR.SP00.A,EXR.D.CNY.EUR.SP00.A,EXR.D.CYP.EUR.SP00.A,EXR.D.CZK.EUR.SP00.A,EXR.D.DKK.EUR.SP00.A,...,EXR.D.RON.EUR.SP00.A,EXR.D.RUB.EUR.SP00.A,EXR.D.SEK.EUR.SP00.A,EXR.D.SGD.EUR.SP00.A,EXR.D.SIT.EUR.SP00.A,EXR.D.SKK.EUR.SP00.A,EXR.D.THB.EUR.SP00.A,EXR.D.TRY.EUR.SP00.A,EXR.D.USD.EUR.SP00.A,EXR.D.ZAR.EUR.SP00.A
1,Unité :,Dollar Australien (AUD),Lev Nouveau (BGN),Real Bresilien (BRL),Dollar Canadien (CAD),Franc Suisse (CHF),Yuan Ren Min Bi (CNY),Livre Cypriote (CYP),Couronne Tcheque (CZK),Couronne Danoise (DKK),...,Nouveau Ron (RON),Rouble Russe (RUB) (RUB),Couronne Suedoise (SEK),Dollar de Singapour (SGD),Tolar (SIT),Couronne Slovaque (SKK),Baht (THB),Nouvelle Livre Turque (TRY),Dollar des Etats-Unis (USD),Rand (ZAR)
2,Magnitude :,Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),...,Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0)
3,Méthode d'observation :,Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),...,Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A)
4,Source :,BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),...,BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0)


In [230]:
# With the index, we can drop the rows for multiple dates
df_Webstat.drop([0,1,2,3,4], inplace = True)
df_Webstat.head()

,dates,Dollar australien (AUD),Lev bulgare (BGN),Real brésilien (BRL),Dollar canadien (CAD),Franc suisse (CHF),Yuan renminbi chinois (CNY),Livre chypriote (CYP),Couronne tchèque (CZK),Couronne danoise (DKK),...,Leu roumain (RON),Rouble russe (RUB),Couronne suédoise (SEK),Dollar de Singapour (SGD),Tolar slovène (SIT),Couronne slovaque (SKK),Baht thaïlandais (THB),Livre turque (TRY),Dollar des Etats-Unis (USD),Rand sud-africain (ZAR)
5749,29/12/2006,1.6691,1.9558,2.8141,1.5281,1.6069,10.2793,0.5782,27.485,7.456,...,3.3835,34.68,9.0404,2.0202,239.64,34.435,46.77,1.864,1.317,9.2124
5750,28/12/2006,1.6699,1.9558,2.8175,1.5268,1.6058,10.2935,0.5782,27.54,7.4573,...,3.398,34.694,9.0463,2.0209,239.64,34.561,47.252,1.8648,1.3173,9.259
5751,27/12/2006,1.6765,1.9558,2.8343,1.5297,1.6058,10.2923,0.5782,27.59,7.4543,...,3.3695,34.666,9.023,2.0217,239.65,34.349,47.776,1.8751,1.3159,9.2198
5752,26/12/2006,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5753,25/12/2006,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [87]:
pd.to_datetime?

In [231]:
# conversion en dates
df_Webstat['dates'] = pd.to_datetime(df_Webstat['dates'])

df_Webstat['dates'].dtypes

dtype('<M8[ns]')

In [232]:
# Trier le DataFrame par date
df_Webstat.sort_values(by='dates', inplace=True)
df_Webstat.head()

,dates,Dollar australien (AUD),Lev bulgare (BGN),Real brésilien (BRL),Dollar canadien (CAD),Franc suisse (CHF),Yuan renminbi chinois (CNY),Livre chypriote (CYP),Couronne tchèque (CZK),Couronne danoise (DKK),...,Leu roumain (RON),Rouble russe (RUB),Couronne suédoise (SEK),Dollar de Singapour (SGD),Tolar slovène (SIT),Couronne slovaque (SKK),Baht thaïlandais (THB),Livre turque (TRY),Dollar des Etats-Unis (USD),Rand sud-africain (ZAR)
8090,2000-01-08,1.5965,1.9461,1.6504,1.3794,1.546,7.6678,0.57359,35.421,7.4575,...,2.0315,25.8276,8.474,1.605,208.3626,42.505,38.0859,0.5886,0.9264,6.4473
8059,2000-01-09,1.5534,1.9463,1.6233,1.3102,1.55,7.3682,0.57241,35.31,7.4579,...,2.0519,24.7242,8.3795,1.5334,208.9769,42.618,36.4554,0.5823,0.8902,6.2099
8029,2000-01-10,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7998,2000-01-11,1.6341,1.9464,1.6321,1.3038,1.5279,7.0801,0.57241,34.793,7.4434,...,2.1261,23.8292,8.474,1.494,210.4025,43.625,37.3746,0.5827,0.8554,6.4255
7968,2000-01-12,1.6302,1.9494,1.7295,1.3361,1.5134,7.23,0.57241,34.643,7.456,...,2.2151,24.352,8.6825,1.5299,212.3641,43.233,38.2549,0.6004,0.8735,6.7041


In [251]:
# use dt.strftime if you need to convert datetime to other formats (but note that then dtype of column will be object (string))
#df_Webstat['dates'] = df_Webstat['dates'].dt.strftime("%d/%m/%Y")
#df_Webstat['dates'].dtypes


In [234]:
# supprimer la colonne dont le nom ne se termine pas par un code ISO3
df_Webstat.drop("Roupie Indienne (100 paise)", axis=1, inplace = True)
# df_Webstat.info()

In [235]:
# changer l'index du DataFrame par la colonne des dates
df_Webstat = df_Webstat.set_index("dates")
df_Webstat.head()

,Dollar australien (AUD),Lev bulgare (BGN),Real brésilien (BRL),Dollar canadien (CAD),Franc suisse (CHF),Yuan renminbi chinois (CNY),Livre chypriote (CYP),Couronne tchèque (CZK),Couronne danoise (DKK),Couronne estonienne (EEK),...,Leu roumain (RON),Rouble russe (RUB),Couronne suédoise (SEK),Dollar de Singapour (SGD),Tolar slovène (SIT),Couronne slovaque (SKK),Baht thaïlandais (THB),Livre turque (TRY),Dollar des Etats-Unis (USD),Rand sud-africain (ZAR)
dates,,,,,,,,,,,,,,,,,,,,,
2000-01-08,1.5965,1.9461,1.6504,1.3794,1.546,7.6678,0.57359,35.421,7.4575,15.6466,...,2.0315,25.8276,8.474,1.605,208.3626,42.505,38.0859,0.5886,0.9264,6.4473
2000-01-09,1.5534,1.9463,1.6233,1.3102,1.55,7.3682,0.57241,35.31,7.4579,15.6466,...,2.0519,24.7242,8.3795,1.5334,208.9769,42.618,36.4554,0.5823,0.8902,6.2099
2000-01-10,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2000-01-11,1.6341,1.9464,1.6321,1.3038,1.5279,7.0801,0.57241,34.793,7.4434,15.6466,...,2.1261,23.8292,8.474,1.494,210.4025,43.625,37.3746,0.5827,0.8554,6.4255
2000-01-12,1.6302,1.9494,1.7295,1.3361,1.5134,7.23,0.57241,34.643,7.456,15.6466,...,2.2151,24.352,8.6825,1.5299,212.3641,43.233,38.2549,0.6004,0.8735,6.7041


In [236]:
# Modifier les noms de colonnes en ne gardant que leur code ISO3: les 4 derniers caractères
df_Webstat.columns = df_Webstat.columns.str[-4:]
df_Webstat.columns = df_Webstat.columns.str[:3] # pour les 3 premiers caractères
df_Webstat.head()

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,RON,RUB,SEK,SGD,SIT,SKK,THB,TRY,USD,ZAR
dates,,,,,,,,,,,,,,,,,,,,,
2000-01-08,1.5965,1.9461,1.6504,1.3794,1.546,7.6678,0.57359,35.421,7.4575,15.6466,...,2.0315,25.8276,8.474,1.605,208.3626,42.505,38.0859,0.5886,0.9264,6.4473
2000-01-09,1.5534,1.9463,1.6233,1.3102,1.55,7.3682,0.57241,35.31,7.4579,15.6466,...,2.0519,24.7242,8.3795,1.5334,208.9769,42.618,36.4554,0.5823,0.8902,6.2099
2000-01-10,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2000-01-11,1.6341,1.9464,1.6321,1.3038,1.5279,7.0801,0.57241,34.793,7.4434,15.6466,...,2.1261,23.8292,8.474,1.494,210.4025,43.625,37.3746,0.5827,0.8554,6.4255
2000-01-12,1.6302,1.9494,1.7295,1.3361,1.5134,7.23,0.57241,34.643,7.456,15.6466,...,2.2151,24.352,8.6825,1.5299,212.3641,43.233,38.2549,0.6004,0.8735,6.7041


In [237]:
# convertir les colonnes donnant le taux de change en flottant
df_Webstat = df_Webstat.apply(pd.to_numeric, errors='coerce')

df_Webstat.head()

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,RON,RUB,SEK,SGD,SIT,SKK,THB,TRY,USD,ZAR
dates,,,,,,,,,,,,,,,,,,,,,
2000-01-08,1.5965,1.9461,1.6504,1.3794,1.5460,7.6678,0.57359,35.421,7.4575,15.6466,...,2.0315,25.8276,8.4740,1.6050,208.3626,42.505,38.0859,0.5886,0.9264,6.4473
2000-01-09,1.5534,1.9463,1.6233,1.3102,1.5500,7.3682,0.57241,35.310,7.4579,15.6466,...,2.0519,24.7242,8.3795,1.5334,208.9769,42.618,36.4554,0.5823,0.8902,6.2099
2000-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-11,1.6341,1.9464,1.6321,1.3038,1.5279,7.0801,0.57241,34.793,7.4434,15.6466,...,2.1261,23.8292,8.4740,1.4940,210.4025,43.625,37.3746,0.5827,0.8554,6.4255
2000-01-12,1.6302,1.9494,1.7295,1.3361,1.5134,7.2300,0.57241,34.643,7.4560,15.6466,...,2.2151,24.3520,8.6825,1.5299,212.3641,43.233,38.2549,0.6004,0.8735,6.7041


In [250]:
# df_Webstat.dtypes

In [248]:
df_Webstat = df_Webstat.dropna()

In [242]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    
    df_Webstat = pd.read_csv('Webstat_Export.csv', sep=';')
    df_Webstat = df_Webstat.apply(lambda x: x.str.replace(',','.'))
    df_Webstat = df_Webstat.dropna()
    df_Webstat = df_Webstat.rename(columns={'Titre :': 'dates'})
    df_Webstat.drop([0,1,2,3,4], inplace = True)
    df_Webstat['dates'] = pd.to_datetime(df_Webstat['dates'])
    df_Webstat.sort_values(by='dates', inplace=True)
    df_Webstat.drop("Roupie Indienne (100 paise)", axis=1, inplace = True)
    df_Webstat = df_Webstat.set_index("dates")
    df_Webstat.columns = df_Webstat.columns.str[-4:]
    df_Webstat.columns = df_Webstat.columns.str[:3] 
    df_Webstat = df_Webstat.apply(pd.to_numeric, errors='coerce')
    df = df_Webstat.dropna()
    
    return df[devises]

In [247]:
# df_taux_change(["CHF", "GBP", "USD"])

### Tests

In [244]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    

In [245]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [246]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.334s

OK
